### scraping b-ok site (name of books , urls) using selenium , then download all books 

In [1]:
#Importing packages
from selenium import webdriver
import pandas as pd
import os 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import pandas as pd
from collections.abc import Iterable
import warnings
# from pymongo import MongoClient
# from collections.abc import Iterable
import time

In [2]:
selenium_driver_path = "C:/chromedriver"

In [3]:
# get unique elements from a list
def unique(data):
    return list(dict.fromkeys(data))

In [4]:
# get selenium driver object
def get_selenium_driver():
    """
    This function returns the selenium driver object.

    Parameters:
        None

    Returns:
        driver: selenium driver object
    """

    driver = webdriver.Chrome(executable_path = selenium_driver_path)

    return driver

In [5]:
# get BeautifulSoup object
def get_soup(url):
    """
    Given the url of a page, this function returns the soup object.
    
    Parameters:
        url: the link to get soup object for
    
    Returns:
        soup: soup object
    """
    driver = get_selenium_driver()

    driver.get(url)

    driver.implicitly_wait(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    driver.close()

    return soup

In [6]:
#save to MongoDB
def saveToDB(collection, city_areas_zones_dict):
    
    try:
        print("collection.count_documents({}): ",  collection.count_documents({}))
        resultSet = collection.count_documents({"City":city_areas_zones_dict["City"],"Areas_Zones":city_areas_zones_dict['Areas_Zones']})

        if resultSet > 0:
            print("city_areas_zones_dict is already scrapped >> ", city_areas_zones_dict)
            print("Skip!")
        else:
            print("New city_areas_zones_dict will be scrapped >> ", city_areas_zones_dict)
            print("I will insert it!")
            collection.insert_one(city_areas_zones_dict)
            
    except Exception as e:
        print('Exception : %s' % str(e))

In [7]:
import re

# scrape b-ok website

def scrape_books():
    books_names = []

    for num in range(1,2):
        base_url = "https://b-ok.cc/s/natural%20language%20processing?page="+str(num)

        home_page_soup = get_soup(base_url)

        # get a get_selenium_driver object
        driver = get_selenium_driver()
        #driver.execute_script("return navigator.userAgent")
        driver.get(base_url)
        

        home_page_soup=home_page_soup.find_all("div",{'id':"searchResultBox"})
        links=home_page_soup[0].find_all("a",{"href":re.compile("^/book/")})
        #print(links)
            

        all_link = []
        for link in links:
            part_of_link = link.get("href")
            link = "https://b-ok.cc"+part_of_link
            all_link.append(link)
            all_link = list(set(all_link))
            print("all_link")
            print(len(all_link))

            print(all_link)
        
        for link in all_link:
            page_driver2 = get_selenium_driver()
            page_driver2.get(link)
            page_driver2.find_element_by_class_name("dlButton").click()
            


        
        time.sleep(3)
        print("----------------------------------")
    driver.close()
        


In [8]:
scrape_books()

all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3642068/da7d8c']
all_link
2
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3642068/da7d8c']
all_link
3
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3711505/debacd']
all_link
3
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3711505/debacd']
all_link
4
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3711505/debacd']
all_link
4
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3']
all_link
5
['https://b-ok.cc/book/3681492/07d64c', 'https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

# Read Table of Content

In [9]:
#installing pdfminer
!pip install pdfminer

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

In [11]:
def find_last_level(list_of_tuples,level):
    """" this function gets the last title of a certain level
    
     parameters:
         list_of_tuples: is a list of tubles each tuple consists of (level,title)
         level: is the level that we are tying to get the last title of from the list of tuples
         
     return:
         title: is the last title of the chosen level"""
    
    list_of_levels = []
    for i in list_of_tuples:
        if i[0] == level:
            list_of_levels.append(i)
    title = list_of_levels[-1][1]
    return title
        

In [12]:
def search_values(level,dct,main_level,last,title):
    """"this function uses recurssion to find the parent of a certain title
    
    parameters:
        level: is the level-1 of the title we are looking for its parent we use this parameter for recurssion
        dct: is the dictionry we are searching in
        main_level: is the same as level parameter but used to save the first value of level in the case of recurrsion
        last: list of tuples consists of [(level,title)]
        title: the main title we are looking for it's parent
        
    return:
        dct: the updated dictionary"""
    
    if find_last_level(last,level) in list(dct.keys()):
        if find_last_level(last,level) != find_last_level(last,main_level):
            if type(dct[find_last_level(last,level)][-1]) is dict:
                new_dct = dct[find_last_level(last,level)][-1]
                dct[find_last_level(last,level)][-1] = search_values(main_level,new_dct,main_level,last,title)
                return dct
            else:
                dct[find_last_level(last,level)][-1] = {str(dct[find_last_level(last,level)][-1]):[title]}
                return dct
        else:
            dct[find_last_level(last,level)].append(title)
            return dct
    else:
        return search_values(level-1,dct,main_level,last,title)
            

In [13]:
# Open a PDF document.
fp = open("Natural Language Processing and Text Mining by Anne Kao, Steve R. Poteet (z-lib.org).pdf", 'rb')

parser = PDFParser(fp)
document = PDFDocument(parser)
dct = {}
last = []             
# Get the outlines of the document.
outlines = document.get_outlines()
for (level,title,dest,a,se) in outlines:
    if level == 1:
        last.append((level,title))
        dct[title] = []
    elif level > 1:
        last.append((level,title))
        search_values(level-1,dct,level-1,last,title)

dct

{'Contents': [],
 '1 Overview': [],
 '2 Extracting Product Features and Opinions from Reviews': [],
 '3 Extracting Relations from Text: From Word Sequences to Dependency Paths': [],
 '4 Mining Diagnostic Text Reports by Learning to Annotate Knowledge Roles': [],
 '5 A Case Study in Natural Language Based Web Search': [],
 '6 Evaluating Self-Explanations in iSTART: Word Matching, Latent Semantic Analysis, and Topic Models': [],
 '7 Textual Signatures: Identifying Text-Types Using Latent Semantic Analysis to Measure the Cohesion of Text Structures': [],
 '8 Automatic Document Separation: A Combination of Probabilistic Classification and Finite-State Sequence Modeling': [],
 '9 Evolving Explanatory Novel Patterns for Semantically-Based Text Mining': [],
 '10 Handling of Imbalanced Data in Text Classification: Category-Based Term Weights': [],
 '11 Automatic Evaluation of Ontologies': [],
 '12 Linguistic Computing with UNIX Tools': [],
 'Index': ['A',
  'B',
  'C',
  'D',
  'E',
  'F',
  '

# Print Table of Content Before Summarization

In [14]:
for sentence in dct.keys():
    print(sentence)

Contents
1 Overview
2 Extracting Product Features and Opinions from Reviews
3 Extracting Relations from Text: From Word Sequences to Dependency Paths
4 Mining Diagnostic Text Reports by Learning to Annotate Knowledge Roles
5 A Case Study in Natural Language Based Web Search
6 Evaluating Self-Explanations in iSTART: Word Matching, Latent Semantic Analysis, and Topic Models
7 Textual Signatures: Identifying Text-Types Using Latent Semantic Analysis to Measure the Cohesion of Text Structures
8 Automatic Document Separation: A Combination of Probabilistic Classification and Finite-State Sequence Modeling
9 Evolving Explanatory Novel Patterns for Semantically-Based Text Mining
10 Handling of Imbalanced Data in Text Classification: Category-Based Term Weights
11 Automatic Evaluation of Ontologies
12 Linguistic Computing with UNIX Tools
Index


# Print Table of Content After Summarization by LexRankSummarizer

In [15]:
## Using SUmy package
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [16]:
for sentence in dct.keys():
    #print(eachsentence)
    # For Strings
    parser=PlaintextParser.from_string(sentence,Tokenizer("english"))
    # Using LexRank
    summarizer = LexRankSummarizer()
    #Summarize each sentence with 1 sentences
    summary = summarizer(parser.document,1)
    for sentence in summary:
        print(sentence)
   # print(" ")

Contents
1 Overview
2 Extracting Product Features and Opinions from Reviews
3 Extracting Relations from Text: From Word Sequences to Dependency Paths
4 Mining Diagnostic Text Reports by Learning to Annotate Knowledge Roles
5 A Case Study in Natural Language Based Web Search
6 Evaluating Self-Explanations in iSTART: Word Matching, Latent Semantic Analysis, and Topic Models
7 Textual Signatures: Identifying Text-Types Using Latent Semantic Analysis to Measure the Cohesion of Text Structures
8 Automatic Document Separation: A Combination of Probabilistic Classification and Finite-State Sequence Modeling
9 Evolving Explanatory Novel Patterns for Semantically-Based Text Mining
10 Handling of Imbalanced Data in Text Classification: Category-Based Term Weights
11 Automatic Evaluation of Ontologies
12 Linguistic Computing with UNIX Tools
Index


# Result or Conclusion:LexRankSummarizer algorithm is not good in text summarization as it needs a lot of sentences to get the most important ones from them and return them back , but doesnot summarize one single sentence 

# How to solve problem: i have used spacy package to brake up each sentence into smaller sentences as much as possible to be able to work with lexRnakSummarizer approach

In [35]:
#importing needed packages
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
import en_core_web_sm
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [36]:
model="en_core_web_sm"

#import en_core_web_sm
nlp = en_core_web_sm.load()

large_list=list()
for i in dct.keys(): #break down each sentence into smaller ones
    doc2 = nlp(i)
    sentences = [x for x in doc2.sents]
    small_list=list()
    for y in sentences:
        small_list.append(y)
    large_list.append(small_list) 

In [37]:
#merge each small sentences with each other again to form one sentence, and put a 
#fullstop at the end of each sentence so the algorithm can regognize 
#that smaller sentences are not in one sentence any more and trait them as a different sentences

final_list=list() 
for i in large_list:
    #print(i)
    #print(" ")
    whole_sentence=""
    sentence=""
    for e in i:
        #print(e)
        whole_sentence=whole_sentence+str(e)+". "
    final_list.append(whole_sentence)

In [38]:
#apply LexRankSummarizer on each sentence in the table of content 

summary_list=list()
for eachSentence in final_list:
    #print(eachsentence)
    # For Strings
    parser=PlaintextParser.from_string(eachSentence,Tokenizer("english"))
    # Using LexRank
    summarizer = LexRankSummarizer()
    #Summarize each sentence with 1 sentences
    summary = summarizer(parser.document,1)
    #summary_list=summary
    for sentence in summary:
        #print(sentence)
        summary_list.append(sentence)

In [39]:
#clean the summarized sentences
list_to_print=list()
for eachSentence in summary_list:
    #print(eachSentence)
    eachSentence=str(eachSentence).replace('.', '')
    eachSentence=str(eachSentence).replace(':', '')
    list_to_print.append(str(eachSentence))
    #print(eachSentence)

# Print Table of Content Before Summarization and Resolving Algorithm's Limitation

In [40]:
for sentence in dct.keys():
    print(sentence)

Contents
1 Overview
2 Extracting Product Features and Opinions from Reviews
3 Extracting Relations from Text: From Word Sequences to Dependency Paths
4 Mining Diagnostic Text Reports by Learning to Annotate Knowledge Roles
5 A Case Study in Natural Language Based Web Search
6 Evaluating Self-Explanations in iSTART: Word Matching, Latent Semantic Analysis, and Topic Models
7 Textual Signatures: Identifying Text-Types Using Latent Semantic Analysis to Measure the Cohesion of Text Structures
8 Automatic Document Separation: A Combination of Probabilistic Classification and Finite-State Sequence Modeling
9 Evolving Explanatory Novel Patterns for Semantically-Based Text Mining
10 Handling of Imbalanced Data in Text Classification: Category-Based Term Weights
11 Automatic Evaluation of Ontologies
12 Linguistic Computing with UNIX Tools
Index


# Print Table of Content After Summarization and Resolving Algorithm's Limitation

In [41]:
list_to_print

['Contents',
 '1 Overview',
 '2 Extracting Product Features and Opinions from Reviews',
 '3 Extracting Relations from Text From Word Sequences to Dependency Paths',
 '4 Mining Diagnostic Text Reports by Learning to Annotate Knowledge Roles',
 '5 A Case Study in Natural Language',
 '6 Evaluating Self-Explanations in iSTART',
 '7 Textual Signatures',
 '8 Automatic Document Separation',
 '9 Evolving Explanatory Novel Patterns for Semantically-Based Text Mining',
 '10 Handling of Imbalanced Data in Text Classification Category-Based Term Weights',
 '11 Automatic Evaluation of Ontologies',
 '12 Linguistic Computing with UNIX Tools',
 'Index']